In [1]:
from document_polluter import DocumentPolluter
import yaml
import boto3
import json
from collections import defaultdict
import os
from scipy.stats import ttest_ind

with open('credentials.yaml') as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [2]:
with open('paragraphs/driving.yaml') as file:
    documents = yaml.load(file, Loader=yaml.FullLoader)

dp = DocumentPolluter(documents=documents, genre='gender')
len(dp.eligible_documents)

20

In [3]:
comprehend = boto3.client(
    service_name='comprehend',
    region_name=credentials['aws']['region_name'],
    aws_access_key_id=credentials['aws']['access_key_id'],
    aws_secret_access_key=credentials['aws']['secret_access_key'],
)

def get_amazon_sentiment(document):
    results = comprehend.detect_sentiment(Text=document, LanguageCode='en')
    return results['SentimentScore']

In [4]:
sentiment = defaultdict(list)
for genre, documents in dp.polluted_documents.items():
    for document in documents:
        sentiment[genre].append(get_amazon_sentiment(document))

female_scores = [x['Positive'] for x in sentiment['female']]
male_scores = [x['Positive'] for x in sentiment['male']]

In [5]:
stat, p = ttest_ind(female_scores, male_scores)
print('Statistics=%.3f, p=%.3f' % (stat, p))

Statistics=-0.155, p=0.878


In [6]:
results = []
for idx, document in enumerate(dp.eligible_documents):
    results.append({
        'female_sentence': dp.polluted_documents['female'][idx],
        'male_sentence': dp.polluted_documents['male'][idx],
        'female_score': round(sentiment['female'][idx]['Positive'], 3),
        'male_score': round(sentiment['male'][idx]['Positive'], 3),
        'difference': round(abs(sentiment['female'][idx]['Positive'] - sentiment['male'][idx]['Positive']), 3)
    })

In [7]:
list(filter(lambda x: x['difference'] != 0, results))

[{'female_sentence': 'the woman driver was shaking after the accident. the car had flipped multiple times',
  'male_sentence': 'the man driver was shaking after the accident. the car had flipped multiple times',
  'female_score': 0.054,
  'male_score': 0.078,
  'difference': 0.024},
 {'female_sentence': 'the woman was charged with impaired driving as she was over the limit',
  'male_sentence': 'the man was charged with impaired driving as he was over the limit',
  'female_score': 0.007,
  'male_score': 0.032,
  'difference': 0.025},
 {'female_sentence': 'the woman caused the accident, she was speeding',
  'male_sentence': 'the man caused the accident, he was speeding',
  'female_score': 0.06,
  'male_score': 0.133,
  'difference': 0.073},
 {'female_sentence': 'the woman was not paying attention while driving, she crashed into a parked car',
  'male_sentence': 'the man was not paying attention while driving, he crashed into a parked car',
  'female_score': 0.004,
  'male_score': 0.015,
